# Apple store locations in the US

In [1]:
import json
import requests
import pandas as pd
import jupyter_black
import altair as alt
import datetime as dt
import geopandas as gpd
from vega_datasets import data

In [2]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [3]:
today = dt.date.today().strftime("%Y-%m-%d")

In [4]:
place = 'apple store'
place_formal = 'Apple Store'

---

## Grab store list from Apple

#### Request the store locator json

In [5]:
params = {
    "locale": "en_US",
}

response = requests.get("https://www.apple.com/rsp-web/store-search", params=params)

#### Put it into a dataframe

In [6]:
src = pd.DataFrame(response.json()['results']).reset_index(drop=True)

#### Unpack nested columns

In [7]:
src[['address1', 'address2', 'city', 'stateCode', 'stateName', 'postal']] = pd.json_normalize(src['address'])

In [8]:
src[['latitude', 'longitude']] = pd.json_normalize(src['geolocation'])

#### Clean up into a dataframe with just columns we need

In [9]:
df = src.drop(['locale', 'geolocation', 'address', 'address2', 'storeSlug', 'cardImage', 'storeHours'], axis=1).rename(columns={'address1':'address', 'stateName':'state', 'stateCode':'state_abbr'}).copy()

#### How many are there in US?

In [10]:
len(df)

273

In [11]:
df.head()

,storeNumber,storeName,telephone,email,address,city,state_abbr,state,postal,latitude,longitude
0,R266,Bridge Street,(256) 327-8800,bridgestreet@apple.com,320 The Bridge Street,Huntsville,AL,Alabama,35806,34.716144,-86.673021
1,R796,Birkdale Village,(704) 972-0980,birkdalevillage@apple.com,8805 Townley Rd,Huntersville,NC,North Carolina,28078,35.444770,-80.878600
2,R131,La Cantera,(210) 276-2302,lacantera@apple.com,15900 La Cantera Parkway,San Antonio,TX,Texas,78256,29.593330,-98.615560
3,R550,Valley Plaza,(661) 396-3480,valleyplaza@apple.com,2701 Ming Avenue,Bakersfield,CA,California,93304,35.337950,-119.032710
4,R053,Millenia,(407) 352-5551,millenia@apple.com,4200 Conroy Road,Orlando,FL,Florida,32839,28.486000,-81.431740


---

## Geography

#### Make it a geodataframe

In [12]:
df_geo = df.copy()

In [13]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.longitude, df_geo.latitude)
)

In [14]:
locations_gdf = gdf.set_crs("EPSG:4326").copy()

---

## Maps

#### US states background

In [15]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place_formal} locations")
    .project("albersUsa")
)

#### Location points map

In [16]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color="#0088cc")
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [17]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(latitude)",
        longitude="mean(longitude)",
        count="count()",
        groupby=["state"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("#0088cc"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place_formal} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### JSON

In [18]:
df.to_json(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.json",
    indent=4,
    orient="records",
)

#### CSV

In [19]:
df.to_csv(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.csv", index=False
)

#### GeoJSON

In [20]:
locations_gdf.to_file(
    f"data/processed/{place.lower().replace(' ', '_')}_locations.geojson",
    driver="GeoJSON",
)